<a href="https://colab.research.google.com/github/RafaelCaballero/Julio24/blob/main/code/06ejemplodataframes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Introducción a la ciencia de datos con Python
###  Rafa Caballero

# Caso práctico: análisis básico de datos, gráficos con pandas

Comenzando cargando y preparando los datos con los que vamos a trabajar

In [ ]:
import yfinance as yf
import pandas as pd

# Lista de tickers:
valores = ["inditex","iberdrola", "santander", "BBVA", "naturgy"]
tickers = ['ITX.MC', 'IBE.MC',    'SAN.MC', 'BBVA.MC', 'NTGY.MC']

# Fechas de inicio y fin
start_date = '2024-01-01'
end_date = '2024-07-02'

# Descargar los datos de Yahoo Finance
data = yf.download(tickers, start=start_date, end=end_date, group_by='ticker')

# Crear un DataFrame vacío para almacenar los valores OPEN
open_data = pd.DataFrame()

# Extraer los valores OPEN de cada ticker y añadirlos al DataFrame
for i,ticker in enumerate(tickers):
    open_data[valores[i]] = data[ticker]['Open']


# incremento en porcentaje con respecto al día anterior
roi = open_data.pct_change()*100

# añadimos el día de la semana
roi['dia_semana'] = roi.index.to_series().dt.day_name()

# mejor en castellano
days_translation = {
    'Monday': 'Lunes',
    'Tuesday': 'Martes',
    'Wednesday': 'Miércoles',
    'Thursday': 'Jueves',
    'Friday': 'Viernes',
    'Saturday': 'Sábado',
    'Sunday': 'Domingo'
}
roi['dia_semana'] = roi['dia_semana'].map(days_translation)

# Establecer el índice como una versión formateada de la fecha
roi.index = pd.to_datetime(open_data.index).strftime('%Y-%m-%d')


roi.style.background_gradient(cmap='RdYlGn')

**Ejercicio**

La primera fila tiene valores NaN porque no existe fila anterior. Generar un nuevo dataframe "roi2" que sea igual que "roi" pero quitando la primera fila (idea: usar `roi.iloc[fila]` para seleccionar todo menos la primera fila)

In [ ]:
# solución


# para probar
roi2.style.background_gradient(cmap='RdYlGn')

**Ejercicio**

Queremos obtener el valor de BBVA el 2024-06-28 (ayuda: utilizar `roi2.loc[fila,columna]`)

**Ejercicio**

¿Y si queremos mostrar toda la fila correspondiente a esta fecha?

### Información básica

In [ ]:
roi2.info()

In [ ]:
roi2.describe()

Como se puede ver no muestra información sobre la columna "dia_semana" por no ser numérica. En estos casos se suele comprobar los valores que tiene y su frecuencia

In [ ]:
roi2.dia_semana.unique()

In [ ]:
roi2.dia_semana.value_counts()

### Gráficos sencillos con Pandas

Unos histogramas burdos pero rápidos de las columnas numéricas

In [ ]:
roi2.hist()

Los boxplots dan otra perspectiva

In [ ]:
roi2.boxplot()

In [ ]:
roi2.plot()

uff...no se ve nada...ajustemos un poco el gráfico

In [ ]:
ax = roi2.plot(figsize=(20, 4), title='Evolución de los valores OPEN')
# Configurar etiquetas del eje X para mostrar 1 de cada 5 fechas
tick_positions = roi2.index[::5]
ax.set_xticks(tick_positions)
ax.set_xticklabels(tick_positions.strftime('%Y-%m-%d'), rotation=45)
print("")

¿No se vería mejor con la media móvil?

In [ ]:
# Calcular la media móvil de 5 días
rolling_average = roi2[valores].rolling(window=5).mean()
rolling_average

In [ ]:
ax = rolling_average.plot(figsize=(20, 4), title='Evolución de los valores OPEN, media móvil de 5 días')
# Configurar etiquetas del eje X para mostrar 1 de cada 5 fechas
tick_positions = rolling_average.index[::5]
ax.set_xticks(tick_positions)
ax.set_xticklabels(tick_positions.strftime('%Y-%m-%d'), rotation=45)
print("")

Puede ser que nos interese ver un día concreto en forma de barras:

In [ ]:
roi2.loc[["2024-06-11"]].plot.bar()

In [ ]:
open_data.plot.area()

In [ ]:
roi2.plot.kde() # diagrama de densidad

In [ ]:
roi2.plot.scatter(x="santander",y="BBVA")

In [ ]:
roi2.plot.scatter(x="santander",y="inditex")

In [ ]:
roi2[valores].corr()

Hay muchos más diagramas, ver aquí: https://pandas.pydata.org/docs/user_guide/visualization.html


#### Filtros

**Ejercicio**

En qué día tuvo naturgy su mayor decremento porcentual con respecto al día anterior (el valor mínimo en `roi2`)